In [1]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.functions import sha2, concat_ws, col, concat, when, length, lpad, to_date

In [2]:
#function to read later

def control_null (dataframe): 
    for dfcol in dataframe.columns:
        df_null = dataframe.filter(col(dfcol).isNull()).count()
        if df_null > 0:
            print(f"{dfcol} : {df_null} null")
        else:
            print(f"{dfcol} no tiene null")

def create_hash(Datafram):
    value = Datafram.withColumn("hash_id", sha2(concat_ws("||", *Datafram.columns), 256))
    return value

def write_DB (parquet_DF, table):
    try:
        parquet_DF.write.jdbc(
            url=jdbc_url,
            table=table,
            mode="overwrite",
            properties=properties
        )
    except Exception as error:
        logger.error("Error ruta .yaml:" + str(error))


In [3]:

spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .getOrCreate()
)

spark.sparkContext.setLogLevel("INFO")
logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)



logger.info("Log de ejemplo guardado en archivo y consola.")

errores_detectados = []

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


00:58:59.989 [Thread-3] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [4]:
try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))
    
try:
    read_parquet_airline = config["Parquet_file"]["df_airline"]
    read_parquet_flights = config["Parquet_file"]["df_flights"]
    read_parquet_airports = config["Parquet_file"]["df_airports"]
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))

In [ ]:
#BD conection

try:
    jdbc_url = "jdbc:postgresql://localhost:5433/sparkdb_dev"
    properties = {
        "user": config[sparkdb_dev][user],
        "password": config[sparkdb_dev][password],
        "driver": "org.postgresql.Driver"
    }
except Exception as error:
    logger.error("Error conection BD:" + str(error))

00:59:00.022 [Thread-3] ERROR __main__ - Error conection BD:name 'sparkdb_dev' is not defined


In [6]:
try:
    df_flights = spark.read.parquet(read_parquet_flights)
    df_airline = spark.read.parquet(read_parquet_airline)
    df_airports = spark.read.parquet(read_parquet_airports)
except Exception as error:
    logger.error("Error read.parquet:" + str(error))

00:59:00.205 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
00:59:00.215 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Warehouse path is 'file:/home/illidan/proyecto_desde0/ETL/spark-warehouse'.
00:59:00.237 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@524041c0{/SQL,null,AVAILABLE,@Spark}
00:59:00.239 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@12054ca3{/SQL/json,null,AVAILABLE,@Spark}
00:59:00.240 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@36bdd607{/SQL/execution,null,AVAILABLE,@Spark}
00:59:00.242 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@7034f0ba{/SQL/execution/json,null,AVAILABLE,@Spark}
00:59:00.258 [Threa

00:59:02.862 [Executor task launch worker for task 0.0 in stage 0.0 (TID 0)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 0.0 (TID 0). 3053 bytes result sent to driver
00:59:02.873 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 0.0 (TID 0) in 509 ms on 172.23.57.81 (executor driver) (1/1)
00:59:02.874 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 0.0, whose tasks have all completed, from pool 
00:59:02.881 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ResultStage 0 (parquet at NativeMethodAccessorImpl.java:0) finished in 0.743 s
00:59:02.885 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
00:59:02.885 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Killing all running tasks in stage 0: Stage finish

In [7]:
#combine the year, month and day columns into a single date column
try:
        df_flights = df_flights.withColumn("DATE", 
                                                to_date(
                                                        concat(
                                                                col("YEAR"),
                                                                when(length(col("MONTH")) == 1, lpad(col("MONTH"), 2, "0")).otherwise(col("MONTH")),
                                                                when(length(col("DAY")) == 1, lpad(col("DAY"), 2, "0")).otherwise(col("DAY"))
                                                        ),\
                                                        "yyyyMMdd"
                                                )
                                        )
except Exception as error:
    logger.error("Error al crear la columna DATE:" + str(error))

In [8]:
#Create hash_id to identify the rows when insert into the database
try:
    df_airline = create_hash(df_airline)
    df_airports = create_hash(df_airports)
    df_flights = create_hash(df_flights)
except Exception as error:
    logger.error("Error create_hash:" + str(error))

In [9]:
#charge the data into the database


#write_DB(avg_flight, "avg_flight")

In [10]:


df_airline.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- hash_id: string (nullable = true)

